In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
!pip install torchmetrics
import os
import json

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import Accuracy
from easydict import EasyDict


# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


데이터들을 npy 형태로 불러옴

In [90]:
data = np.load('drive/MyDrive/monitoring_dataset_distributed.npy')

train_x = data["train_x"]
train_y = data["train_y"]
test_x = data["test_x"]

In [91]:
type(train_x)

numpy.ndarray

In [92]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, random_state = 14)

In [93]:
x_train, y_train, x_valid, y_valid = map(torch.tensor, (x_train, y_train, x_valid, y_valid))


In [94]:
bs = 64

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size = 100)

test_x = torch.tensor(test_x)
test_dataset = TensorDataset(test_x)
test_dl = DataLoader(test_dataset, batch_size=bs, shuffle=False)


#### Model 1 (given model)

In [95]:
cfg = EasyDict({
    'dataset_path': 'drive/MyDrive/monitoring_dataset_distributed.npy',
    'epochs': 100,
    'lr': 0.1,
    'weight_decay': 0.0001,
    'momentum': 0.9,
    'threshold': 0.5,
    'valid_size' : 100
})

In [96]:
class BnLinAct(nn.Module):
    def __init__(self, in_dim, out_dim, activation=nn.ReLU, bias=False):
        super().__init__()
        self.seq = nn.Sequential(
            nn.BatchNorm1d(in_dim),
            nn.Linear(in_dim, out_dim, bias=bias),
            activation(inplace=True),
        )

    def forward(self, x):
        return self.seq(x)

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq = nn.Sequential(
            BnLinAct(3900, 1024),
            nn.Dropout(0.1),
            BnLinAct(1024, 256),
            nn.Dropout(0.1),
            nn.Linear(256, 1),
        )
    def forward(self, x):
        return self.seq(x)

In [97]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = SimpleNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda x: 0.95 ** x, last_epoch=-1)
accuracy_meter = Accuracy(threshold=0.5).to(device)

In [98]:
loss_func = torch.nn.functional.binary_cross_entropy

In [99]:
for epoch in range(cfg.epochs):
    accuracy_meter.reset()
    for i, item in enumerate(train_dl):
        x, y = item[0].to(device), item[1].to(device)
        y_hat = model(x).squeeze(-1)
        loss = criterion(y_hat, y.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy_meter.update(torch.sigmoid(y_hat), y)
    with torch.no_grad():
      for xv, yv in valid_dl:
        valid_loss =  criterion(model(xv).squeeze(-1), yv.float())
    scheduler.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch[{epoch+1}] Acc:{accuracy_meter.compute() * 100:.2f} Valid_loss:{valid_loss/cfg.valid_size}', )

model.eval()
accuracy_meter.reset()
predict_list = list()

# for item in test_dl:
#     x = item[0].to(device)
#     y_hat = model(x).squeeze(-1)

#     predict= (torch.sigmoid(y_hat) > cfg.threshold).float().detach().cpu().tolist()
#     predict_list.extend(predict)
 
# df = pd.DataFrame({'pred': predict_list})
# df.to_csv('submission.csv',header=True, index=True, index_label='idx')

Epoch[10] Acc:89.33 Valid_loss:0.016004126518964767
Epoch[20] Acc:97.00 Valid_loss:0.02253871038556099
Epoch[30] Acc:99.00 Valid_loss:0.025566844269633293
Epoch[40] Acc:99.67 Valid_loss:0.027510009706020355
Epoch[50] Acc:99.67 Valid_loss:0.02828184887766838
Epoch[60] Acc:100.00 Valid_loss:0.027285318821668625
Epoch[70] Acc:99.67 Valid_loss:0.028457431122660637
Epoch[80] Acc:99.67 Valid_loss:0.029017455875873566
Epoch[90] Acc:100.00 Valid_loss:0.029224887490272522
Epoch[100] Acc:100.00 Valid_loss:0.028455136343836784
